In [ ]:
!pip install pandas

In [81]:
import datetime
from datetime import datetime as dt
import pandas as pd

from google.cloud import bigtable
from google.cloud.bigtable import column_family
from google.cloud.bigtable import row_filters, row

In [4]:
INSTANCE_ID = 'big-table-db2'
TABLE_ID = 'plant2'

In [7]:
client = bigtable.Client.from_service_account_json('../../lofty-psyche-328203-949244c34274.json', admin = True)

In [8]:
print('Connecting to database instance')
instance = client.instance(INSTANCE_ID)

Connecting to database instance


In [9]:
print('Creating the {} table'.format(TABLE_ID))
table = instance.table(TABLE_ID)

if not table.exists():
    table.create()
else:
    print("ERROR: Table {} already exists".format(TABLE_ID))

Creating the plant2 table
ERROR: Table plant2 already exists


In [ ]:
print('Creating all family columns')

plant_cfId = 'plant'
customer_cf = table.column_family(plant_cfId)
customer_cf.create()

telemetry_cfId = 'telemetry'
items_cf = table.column_family(telemetry_cfId)
items_cf.create()

In [86]:
df = pd.read_csv('../data/Plant_2_Weather_Sensor_Data.csv')
len(df[0:3])
value = str(df['AMBIENT_TEMPERATURE'][0]).encode()
#row_key = 'plant#{}#{}'.format(df['PLANT_ID'][0], df['DATE_TIME'][0]).encode()
#row_key

In [87]:
print('Writing orders to the table')
rows = []
for line in range(len(df)):
    row_key = 'plant#{}#{}'.format(df['PLANT_ID'][line], df['DATE_TIME'][line]).encode()
    date_time = datetime.datetime.strptime(df['DATE_TIME'][line], '%Y-%m-%d %H:%M:%S')
    row = table.direct_row(row_key)

    row.set_cell(plant_cfId, 'source_key'.encode(), df['SOURCE_KEY'][line], timestamp = date_time)

    row.set_cell(telemetry_cfId, 'ambient_temp'.encode(), str(df['AMBIENT_TEMPERATURE'][line]), timestamp = date_time)
    row.set_cell(telemetry_cfId, 'module_temp'.encode(), str(df['MODULE_TEMPERATURE'][line]), timestamp = date_time)
    row.set_cell(telemetry_cfId, 'irradiation'.encode(), str(df['IRRADIATION'][line]), timestamp = date_time)

    rows.append(row)

table.mutate_rows(rows)

Writing orders to the table


[,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 

In [89]:
print('Getting a single plant by row key')
key = 'plant#4136001#2020-05-15 00:00:00'.encode()

row = table.read_row(key)
print_row(row)

Getting a single plant by row key
Reading data for plant#4136001#2020-05-15 00:00:00:
Column Family plant
	source_key: iq8k7ZNt4Mwm3w0 @2020-05-15 00:00:00+00:00
Column Family telemetry
	ambient_temp: 27.004763700000005 @2020-05-15 00:00:00+00:00
	irradiation: 0.0 @2020-05-15 00:00:00+00:00
	module_temp: 25.0607889 @2020-05-15 00:00:00+00:00



In [88]:
def print_row(row):
    print("Reading data for {}:".format(row.row_key.decode('utf-8')))
    for cf, cols in sorted(row.cells.items()):
        print("Column Family {}".format(cf))
        for col, cells in sorted(cols.items()):
            for cell in cells:
                labels = " [{}]".format(",".join(cell.labels)) \
                    if len(cell.labels) else ""
                print(
                    "\t{}: {} @{}{}".format(col.decode('utf-8'),
                                            cell.value.decode('utf-8'),
                                            cell.timestamp, labels))
    print("")